In [9]:
import folium
import csv
import json
from shapely.geometry import shape
import pandas as pd

# 1. Load Telangana boundary GeoJSON
with open("telangana_ac.geojson", "r", encoding="utf-8") as f:
    telangana_geojson = json.load(f)

# 1.5. Load district boundaries
with open("districts.json", "r", encoding="utf-8") as f:
    districts_geojson = json.load(f)

# 1.5. Load district boundaries
with open("TS_District_Boundary_33_FINAL.geojson", "r", encoding="utf-8") as f:
    new_districts_geojson = json.load(f)

# 2. Compute bounds of Telangana for restricting map view
bounds = folium.GeoJson(telangana_geojson).get_bounds()

# 3. Create base map centered on Telangana and restricted to its bounds with clean tiles
m = folium.Map(
    location=[17.9784, 79.5941],
    zoom_start=7,
    max_bounds=True,
    tiles="CartoDB positron",
    attr='CartoDB'
)

m.fit_bounds(bounds)
Marker_coordinates = []

# 4. Create feature groups for layer control
new_district_layer = folium.FeatureGroup(name="New District Boundaries")
district_layer = folium.FeatureGroup(name="District Boundaries")
constituency_layer = folium.FeatureGroup(name="Constituency Boundaries")

# 5. Add district boundaries to district layer
for feature in districts_geojson["features"]:
    # Generate different colors for each district
    district_id = feature['properties'].get('OBJECTID', 0)
    district_name = feature['properties'].get('D_N', 'Unknown District')
    
    # Create distinct colors using HSV color space
    hue = (district_id * 137.5) % 360  # Golden angle for good distribution
    color = f"hsl({hue}, 70%, 50%)"
    darker_color = f"hsl({hue}, 70%, 30%)"  # Darker version for text
    
    folium.GeoJson(
        feature,
        style_function=lambda x, color=color: {
            "color": color,
            "weight": 3,
            "fillOpacity": 0.1
        }
    ).add_to(district_layer)
    
    # Add district name label at centroid
    try:
        geom = feature["geometry"]
        polygon = shape(geom)
        centroid = polygon.centroid
        lat, lon = centroid.y, centroid.x
        
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(html=f'<div style="font-size:15pt; color:{darker_color}; font-weight:bold; text-shadow:1px 1px white">{district_name}</div>')
        ).add_to(district_layer)
    except Exception as e:
        print(f"Error computing centroid for {district_name}: {e}")

# 5. Add district boundaries to NEW district layer
i=0
for feature in new_districts_geojson["features"]:
    # Generate different colors for each district
    district_id = feature['properties'].get('OBJECTID', i)
    i += 1
    district_name = feature['properties'].get('DISTRICT_N', 'Unknown District')
    
    # Create distinct colors using HSV color space
    hue = (district_id * 137.5) % 360  # Golden angle for good distribution
    color = f"hsl({hue}, 70%, 50%)"
    darker_color = f"hsl({hue}, 70%, 30%)"  # Darker version for text
    
    folium.GeoJson(
        feature,
        style_function=lambda x, color=color: {
            "color": color,
            "weight": 3,
            "fillOpacity": 0.1
        }
    ).add_to(new_district_layer)
    
    # Add district name label at centroid
    try:
        geom = feature["geometry"]
        polygon = shape(geom)
        centroid = polygon.centroid
        lat, lon = centroid.y, centroid.x
        
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(html=f'<div style="font-size:15pt; color:{darker_color}; font-weight:bold; text-shadow:1px 1px white">{district_name}</div>')
        ).add_to(new_district_layer)
    except Exception as e:
        print(f"Error computing centroid for {district_name}: {e}")

# 6. Add constituency boundaries and labels to constituency layer
for feature in telangana_geojson["features"]:
    geom = feature["geometry"]
    props = feature["properties"]
    constituency_name = props.get("assembly_1") or props.get("assembly") or "Unknown"

    folium.GeoJson(
        feature,
        style_function=lambda x: {
            "color": "#0000FF",
            "weight": 0.5,
            "fillOpacity": 0.05
        }
    ).add_to(constituency_layer)

    # Compute centroid using shapely
    try:
        polygon = shape(geom)
        centroid = polygon.centroid
        lat, lon = centroid.y, centroid.x
        Marker_coordinates.append({"constituency": constituency_name, "latitude": lat, "longitude": lon})
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(html=f'<div style="font-size:10pt; color:blue; text-shadow:1px 1px black">{constituency_name}</div>')
        ).add_to(constituency_layer)
    except Exception as e:
        print(f"Error computing centroid for {constituency_name}: {e}")

# 7. Add layers to map
district_layer.add_to(m)
constituency_layer.add_to(m)
new_district_layer.add_to(m)

# 8. Add layer control
folium.LayerControl().add_to(m)

# 5. Load MLA data from CSV
with open("mlas.csv", newline='', encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        try:
            lat = float(row["latitude"])
            lon = float(row["longitude"])
            name = row["name"]
            constituency = row["constituency"]
            party = row["party"]
            photo_url = row["photo_url"]

            # Popup HTML
            popup_html = f"""
            <div style=\"text-align:center; width:180px;\">
                <img src=\"{photo_url}\" style=\"width:100px; height:100px; border-radius:8px; margin-bottom:10px;\"><br>
                <strong>{name}</strong><br>
                <em>{constituency}</em><br>
                <span>{party}</span>
            </div>
            """
            iframe = folium.IFrame(html=popup_html, width=200, height=200)
            popup = folium.Popup(iframe, max_width=250)

            # Add marker
            folium.Marker(
                location=[lat, lon],
                popup=popup,
                icon=folium.Icon(color="blue", icon="user")
            ).add_to(m)
        except Exception as e:
            print(f"Skipping row due to error: {e}")

# 6. Save to HTML file
m.save("telangana_mla_map.html")
#pd.DataFrame(Marker_coordinates).to_csv("telangana_constituency_centroids.csv", index=False)
print("✅ Map saved as telangana_mla_map.html")


Skipping row due to error: could not convert string to float: ''
Skipping row due to error: could not convert string to float: ''
✅ Map saved as telangana_mla_map.html


In [ ]:
import os

# Path to the folder containing the photos
folder_path = "./New folder"  # Change this to your actual path
folder_path1 = "./New folder1"  # Change this to your actual path
# Supported image extensions
image_extensions = (".jpg", ".jpeg", ".png", ".webp")

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(image_extensions):
        # Split filename on comma
        name_part = filename.split(',')[0].strip()

        # Preserve original extension
        ext = os.path.splitext(filename)[1]
        print(ext)
        # Construct new filename
        new_filename = f"{name_part}{'.'}{ext}"
        # Clean and format name
        formatted_name = (
            new_filename.lower()
                .replace(".", "")  # Remove dots like "K. Laxman" -> "K Laxman"
                .replace(" ", "_")  # Replace spaces with underscores
        )
        # Rename the file
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path1, formatted_name)

        os.rename(old_path, new_path)
        print(f"Renamed: {filename} -> {formatted_name}")


Renamed: Dr. Palvai Harish Babu, BJP, AC 1 Sirpur, Komarambheem Asifabad.jpg -> dr_palvai_harish_babujpg
Renamed: SMT, VAKITI SUNITHA LAXMA REDDY, BRS, 37 AC NARSAPUR, MEDAK.jpg -> smtjpg
Renamed: SMT. CHITTEM PARNIKA REDDY, INC, 73 AC NARAYANPET, NARAYANPET.jpg -> smt_chittem_parnika_reddyjpg
Renamed: SMT. KONDA SUREKHA, INC, AC 106 WARANGAL EAST, WARANGAL.jpg -> smt_konda_surekhajpg
Renamed: SMT. MATTA RAGAMAYEE, INC, AC 116 SATHUPALLE (SC), KHAMMAM.jpg -> smt_matta_ragamayeejpg
Renamed: SMT. YASHASWINI MAMIDALA, INC, AC 100 PALAKURTHI, JANAGAM.jpg -> smt_yashaswini_mamidalajpg
Renamed: SMT.ANASUYA SEETHAKKA DANASARI, INC, AC 109 MULUG (ST), MULUGU.jpg -> smtanasuya_seethakka_danasarijpg
Renamed: Smt.Kova Laxmi, BRS, AC 5 Asifabad (ST), KOMARAMBHEEM ASIFABAD.jpg -> smtkova_laxmijpg
Renamed: SMT.PATLOLLA SABITHA INDRA REDDY, BRS, AC 50 MAHESHWARAM, RANGA REDDY.jpg -> smtpatlolla_sabitha_indra_reddyjpg
Renamed: SRI A. REVANTH REDDY, INC, AC 72 KODANGAL, VIKARABAD.jpg -> sri_a_revanth_r

In [3]:
import os

folder_path = "./mla_photos"  # Change to your path
base_url = "https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/"

image_extensions = (".jpg", ".jpeg", ".png", ".webp")

for filename in os.listdir(folder_path):
    if filename.endswith(image_extensions):
        name, ext = os.path.splitext(filename)
        
        # Clean and format name
        formatted_name = (
            name.lower()
                .replace(".", "")  # Remove dots like "K. Laxman" -> "K Laxman"
                .replace(" ", "_")  # Replace spaces with underscores
        )

        # Generate full URL
        image_url = f"{base_url}{formatted_name}{ext.lower()}"
        print(image_url)


https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/dr_palvai_harish_babu.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/smt_chittem_parnika_reddy.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/smt_konda_surekha.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/smt_matta_ragamayee.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/smt_yashaswini_mamidala.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/smtanasuya_seethakka_danasari.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/smt.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/smtkova_laxmi.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/smtpatlolla_sabitha_indra_reddy.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/sri_a_revanth_reddy.jpg
https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/sri_aadi_srinivas.jpg
https://raw.githubusercontent.com/a

In [8]:
import pandas as pd
from rapidfuzz import process, fuzz

# Load the CSV file
df = pd.read_csv("data.csv")

# Get unique values from both columns
col1_values = df['col1'].dropna().unique()
col2_values = df['col2'].dropna().unique()

# Build match dictionary: for each col1 value, find best match in col2
matches = {
    name: process.extractOne(name, col2_values, scorer=fuzz.token_sort_ratio)
    for name in col1_values
}

# Create a new DataFrame with matched results
matched_rows = []
for name, (best_match, score, _) in matches.items():
    matched_rows.append({
        "col1": name,
        "matched_col2": best_match,
        "match_score": score
    })

matched_df = pd.DataFrame(matched_rows)

# Join back to the original dataframe if needed
final_df = pd.merge(df, matched_df, on='col1', how='left')

# Save result
final_df.to_csv("matched_output.csv", index=False)

print(final_df)


                               col1  \
0         Sri Anumula Revanth Reddy   
1            Dr. Palvai Harish Babu   
2                   Smt. Kova Laxmi   
3            Sri Vivek Venkat Swamy   
4                  Sri Gaddam Vinod   
..                              ...   
114               Sri Ramdas Maloth   
115            Smt. Matta Ragamayee   
116      Smt. Chittem Parnika Reddy   
117              Sri Vakiti Srihari   
118  Smt. Anasuya Seethakka Dansari   

                                                  col2  \
0    https://raw.githubusercontent.com/anilreddy488...   
1    https://raw.githubusercontent.com/anilreddy488...   
2    https://raw.githubusercontent.com/anilreddy488...   
3    https://raw.githubusercontent.com/anilreddy488...   
4    https://raw.githubusercontent.com/anilreddy488...   
..                                                 ...   
114  https://raw.githubusercontent.com/anilreddy488...   
115  https://raw.githubusercontent.com/anilreddy488...   
116  http